In [1]:
!pip install transformers
!pip install gensim

You should consider upgrading via the 'c:\users\mohni\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mohni\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Importing required modules


In [2]:
import itertools
import os
from gensim.summarization.bm25 import BM25
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, QuestionAnsweringPipeline
import json
import spacy

In [3]:
class PassageRetrieval:

    def __init__(self, nlp):
        self.tokenize = lambda text: [token.lemma_ for token in nlp(text)]
        self.bm25 = None
        self.passages = None

    def preprocess(self, doc):
        passages = [p for p in doc.split('\n') if p and not p.startswith('=')]
        return passages

    def fit(self, docs):
        # passages = list(itertools.chain(*map(self.preprocess, docs)))
        corpus = [self.tokenize(p) for p in passages]
        self.bm25 = BM25(corpus)
        self.passages = passages

    def most_similar(self, question, topn=10):
        tokens = self.tokenize(question)
        average_idf = sum(map(lambda k: float(self.bm25.idf[k]), self.bm25.idf.keys())) / len(self.bm25.idf.keys())
        scores = self.bm25.get_scores(tokens, average_idf)
        pairs = [(s, i) for i, s in enumerate(scores)]
        pairs.sort(reverse=True)
        passages = [self.passages[i] for _, i in pairs[:topn]]
        return passages

In [4]:
class bert:

    def __init__(self, model):
        self.tokenizer = AutoTokenizer.from_pretrained(model)
        self.model = AutoModelForQuestionAnswering.from_pretrained("/bert_policy_documents")
        self.bert = QuestionAnsweringPipeline(model = self.model, tokenizer = self.tokenizer)
  
    def evaluateAnswer(self, question, sentence):
        answer = self.bert(question = question, context = sentence)
        return answer

In [5]:
def completeAnswer(answer, para):
    for p in para:
        if answer in p:
            return p
    return None

In [6]:
def getPassages(filename):
    with open(filename) as file:
        data = json.load(file)
    passages = list()
    for i in data["vertices"]["paragraphs"]:
        passages.append(i["text"])
    return passages

In [8]:
passages = getPassages("../data/handbook_graph.json")

In [9]:
SPACY_MODEL = os.environ.get("SPACY_MODEL", "en_core_web_sm")
nlp = spacy.load(SPACY_MODEL, disable = ["ner","parser","textcat"])


retreivePassage = PassageRetrieval(nlp)
retreivePassage.fit(passages)
bertModel = bert("deepset/bert-base-cased-squad2")

# questions = [
#              "How do I calculate cgpa",
#              "What is the normal load for UG students",
#              "If I fail a course and take it again in the later semester, will my earlier course with F grade be removed from the transcript",
#             " what is the process of registration?",
#             "how many seats are there in cse for admission?",
#              " what is the admission criteria for btech",
#              "I am in 1st year. Can I take overload?",
#              "I am in 2nd year. Can I take overload?",
#              "what happens if I miss the endsem because of a medical reason?",
#              "what happens if I fail a course?",
#              " what happens if I get an F grade in a course?",
#              "How can I calculate sgpa",
#              "What if I pass all my semesters",
#              "What about canteen",
#              "Will I get hostel",
#              "I dont know anything about IIIT",
#              "Who was abraham lincoln",
#              "Can i take 8 credits of online courses in a semester",
#              "how many credits do i need to graduate",
#              "how is my semester graded",
#              "what if I do more than 156 credits in my btech course"
# ]
questions = [
             "can I take up internships during a semester?",
              "what is the i grade",
              "can I replace a core course on getting an F grade?",
              "how can I get the grade given to me in a course changed?",
              "how will my cgpa be computed if I do more than 156 credits?",
              "is there any rule for attendance?",
              "how can I apply for a semester leave?",
              "how can I apply for branch transfer from ece to cse",
              "what is the minimum credit requirement for graduation?",
              "what are the requirements to get an honours degree?",
              "when is the convocation held?"
]
for q in questions:
  topAnswer = retreivePassage.most_similar(q, topn = 6)
  # print(topAnswer)
  sentence = ""
  for i in topAnswer:
    # print(i)
    sentence += i + " "
  ans = bertModel.evaluateAnswer(q, sentence)
  print(len(sentence.split()))
  print("Q:",q)
  print("Ans:",completeAnswer(ans["answer"], topAnswer))
  # print(len(sentence.split()))
  print("---------------------")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.